In [4]:
import os
os.chdir("/Users/avinashamballa/Desktop/Avinash/Github_Repo/Superposed_Decoding")

In [6]:
pip install fairscale==0.4.13

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 6.7 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fairscale: filename=fairscale-0.4.13-py3-none-any.whl size=332103 sha256=95a890b3d2e29ce9597f564137743421bcff10f832177273a6e252405a88e92b
  Stored in directory: /Users/avinashamballa/Library/Caches/pip/wheels/78/a4/c0/fb0a7ef03cff161611c3fa40c6cf898f76e58ec421b88e8cb3
Successfully built fairscale
Note: you may need to restart the kernel to use updated packages.


In [12]:
%load_ext autoreload
%autoreload 2

import json
import os
import pickle
from datetime import datetime

# import evaluate
import torch
from tqdm import tqdm

from eval import *
from superposed.llama.metrics import *
from superposed.llama.generation import Llama
from superposed.llama.superposed_generation import SuperposedLlama
from superposed.llama.tokenizer import Tokenizer
from superposed.ngrams.ngram_models import make_models

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
sup_device = torch.device("cuda:0")
tokenizer = Tokenizer('7B/tokenizer.model')

In [14]:
# Params
param_file = "params/p15_d3_mixed.json"
with open(param_file, "r") as f:
    params = json.load(f)
    print(f"Parameters: {params}")
alpha = params["alpha"]
temp = params["temp"]
n_drafts = params["n_drafts"]
prompt_len = params["prompt_len"]
n_token_sample = params["n_token_sample"]
i_weights = params["i_weights"]
i_length = params["i_length"]

Parameters: {'alpha': 0.54, 'temp': 0.06, 'n_drafts': 3, 'prompt_len': 15, 'n_token_sample': 9, 'n_token_consider': 32000, 'mixing_method': 'sample_new_weights_with_score', 'smoothing': 'geom', 'sample_tokens': 0, 'sample_beams': 0, 'i_weights': [0.01, 0.04, 0.15, 0.18, 0.12], 'i_length': [1, 2, 3, 4, 5]}


In [15]:
# Create ngram models
ngrams = make_models("ngrams/ckpts-100k", bigram=True, trigram=True, fourgram=True, fivegram=True, sixgram=True, sevengram=False)

Making bigram...


FileNotFoundError: [Errno 2] No such file or directory: 'ngrams/ckpts-100k/b_d_final.pkl'

In [16]:
weight_path = "7B/"
model = SuperposedLlama.build(ckpt_dir=weight_path, 
                         tokenizer_path=f'{weight_path}/tokenizer.model', 
                         max_seq_len=100, 
                         max_batch_size=32,
                         device=sup_device,
                         model_parallel_size=1)

ValueError: Error initializing torch.distributed using env:// rendezvous: environment variable RANK expected, but not set

# Inference

In [17]:
def decode(tokenizer, encoding):
    """
    Args:
        tokenizer (Any): Tokenizer
        encoding (torch.Tensor): Encoding
    Returns:
        decoding (str)
    """
    eos_locs = (encoding == tokenizer.eos_id).nonzero()
    if len(eos_locs > 0):
        encoding = encoding[:eos_locs[0]]
    return tokenizer.decode(encoding.to(torch.int32).tolist())

In [18]:
prompts = [
    "Hi my name is",
    "The Seattle Seahawks were Super Bowl",
    "Penguins are birds native to"
]
tokenized_prompts = tokenizer.encode(prompts, True, False)

In [19]:
alive_gens, _ = model.sup_generate(prompt_tokens=tokenized_prompts, 
                                        smoothing="geom",
                                        max_gen_len=10, 
                                        n_token_sample=n_token_sample,
                                        alpha=alpha, 
                                        temp=temp,
                                        n_drafts=n_drafts,
                                        i_weights=i_weights,
                                        i_length=i_length,
                                        ngrams=ngrams,
                                        get_time=False,
                                        penalty=200)

NameError: name 'model' is not defined

In [24]:
gens = alive_gens[0].reshape(len(prompts) * n_drafts, -1)

In [25]:
for i in gens:
    print(decode(tokenizer, i))

Hi
my name
is L
inda,
I am
a 
40
year old
woman who
